In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import re
from ast import literal_eval
from itertools import combinations
from collections import Counter

#Анализ врачей Москвы: СберЗдоровье и ПроДокторов

**Цель проекта**

Провести анализ данных о врачах Москвы, собранных с онлайн-платформ СберЗдоровье и ПроДокторов, с акцентом на географическое распределение, ценовую политику и рейтинги специалистов.


**Описание источников данных**


**СберЗдоровье** — цифровая платформа, предоставляющая пользователям возможность **записи к врачам**, а также информацию о стоимости приёма, рейтингах и доступности специалистов по районам города.


**ПроДокторов** — **независимый ресурс, ориентированный на отзывы пациентов** и рейтинговую оценку врачей и клиник, что позволяет судить о качестве оказываемых медицинских услуг.


**Этапы работы**

1)Сбор и структурирование данных о врачах Москвы с обеих платформ.

2)Сравнительный анализ стоимости приёма, рейтингов, географии распределения специалистов, анализ текста отзывов.

4)Выявление лучших врачей и районов, где сочетание цены, рейтинга и доступности является наиболее выгодным для пациента.


**MVP - Сервис, который поможет лечиться выгодно**


**Скибиди ДокДок** — прототип, который:


-подсказывает лучшие предложения по рейтингу и цене в выбранном районе;


-показывает средние показатели по Москве;


-помогает пользователю выбрать оптимального врача с учётом репутации, цены и геолокации.


**Долгосрочная цель**


-Построение единого профиля врача с расчётом агрегированного рейтинга, основанного на данных разных платформ.
Для этого необходим детальный анализ различий между источниками — в ценах, рейтингах и опыте специалистов.

ТУТ ОПИСАНИЕ СТРУКТУРЫ НАШИХ ДАТАСЕТОВ И ВСЕХ ПРИЗНАКОВ

#Датасет

In [ ]:
doctors = pd.read_csv('https://github.com/yyaroslavskiy/cuddly-chainsaw/raw/refs/heads/develop/eda/merge/doctors.csv')

In [ ]:
review_rate = pd.read_csv('https://github.com/yyaroslavskiy/cuddly-chainsaw/raw/refs/heads/develop/eda/review_stat.csv')

doctors = doctors.merge(review_rate, left_on='link_sber', right_on='doctor_link', how='left')

doctors = doctors.merge(review_rate, left_on='link_prod', right_on='doctor_link', how='left')

In [ ]:
doctors

In [ ]:
doctors = doctors.drop(columns=['source_x', 'source_y', 'doctor_link_x', 'doctor_link_y'], errors='ignore')
doctors = doctors.rename(columns={
    'rate_x': 'rate_our_sber',
    'rate_y': 'rate_our_prod',
    'comment_x': 'comment_sber',
    'comment_y': 'comment_prod',
})

doctors

In [ ]:
del doctors['Unnamed: 0']

In [ ]:
doctors.info()

In [ ]:
doctors.describe()

посмотрим пропуски(функция с сема)

In [ ]:
#Воспользуемся следующей функцией, которая предоставляет отчет о пропусках в данных

def missing_values_table(df):
    """
    Функция возвращает резюме по пропущенным значениям
    """
    # Общее число пропусков
    mis_val = df.isnull().sum()

    # Процент пропусков
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Создадит таблицу с результатом
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Переименнуем колонки
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})

    # Отсортируем по проценту пропущенных значений
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)

    # Выведем некоторую информацию
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")

    return mis_val_table_ren_columns

missing_values_table(doctors)

Мы сохраняли клиники, чтобы посмотреть в скольких клиниках работает врач. Большинство указывает только одну клинику, но у некоторых есть вторая и третья. Несмотря на то, что в столбцах второй и третьей клиники много пропусков, мы оставляем эти столбцы, так как это важна для анализа продукта (бота) и географии, чтобы привязывать врача к метро и району. Также много пропусков в столбцах с ценой, рейтингом и тд с сайтов сберздоровье и продокторов. Это обусловлено тем, что мы соединяли таблицы, и многих врачей, которых нет на одном сайте, есть на другом. Из-за этого в столбец сайта без информации добавляется Nan.

#Платформа, с которой взяли информацию о враче

FE - добавление колонки источника `'doctor_source'` откуда мы взяли врача

In [ ]:
def get_doctor_source(row):
    has_sber = pd.notna(row['link_sber'])
    has_prod = pd.notna(row['link_prod'])

    if has_sber and has_prod:
        return 'Обе платформы'
    elif has_sber:
        return 'SberHealth'
    elif has_prod:
        return 'ProDoctorov'

doctors['doctor_source'] = doctors.apply(get_doctor_source, axis=1)

In [ ]:
doctors

In [ ]:
doctors['doctor_source'].value_counts()

#ФИО (`name`)

Всего в датасете - 38 865 строк

In [ ]:
doctors.shape[0]

Уникальных фио - 38441


In [ ]:
doctors.name.value_counts().size

230 фио повторяются более одного раза

In [ ]:
print((doctors.name.value_counts() > 1).sum())

Постмотрим на кол-во слов в фио

In [ ]:
doctors.name.str.split().apply(len).value_counts()

In [ ]:
# doctors.name.str.split().apply(len)[doctors.name.str.split().apply(len) == 5]

In [ ]:
doctors

Встречаются лищние \t, но в основном:

- `в случае 2 слов` - это только фамилия имя
- `в случае 4 слов` - это дополнительная фамилия, чаще в скобках. например, фамилия до/после замужества. либо еще встречаются иностранные полные имена
- `в случае 5 слов` - либо несколько фамилий/отчеств, либо иностранные полныеимена

Вручную нашли человека, у которого криво записалось ФИО, вручную поправили

In [ ]:
mask = doctors['name'].fillna('').str.contains(
    r'^\s*орунов\s*\t+\s*евгений\s*\t+\s*михайлович\s*$', case=False, regex=True
)
doctors.loc[mask, 'name'] = 'Орунов Евгений Михайлович'

# Опыт врача (`experience`)

In [ ]:
def categorize_experience(exp):
    if pd.isna(exp):
        return 'No value'
    elif exp <= 5:
        return '0-5 лет'
    elif exp <= 10:
        return '6-10 лет'
    elif exp <= 15:
        return '11-15 лет'
    elif exp <= 20:
        return '16-20 лет'
    elif exp <= 30:
        return '21-30 лет'
    else:
        return 'Более 30 лет'

doctors['experience_category'] = doctors['experience'].apply(categorize_experience)
category_order = ['No value', '0-5 лет', '6-10 лет', '11-15 лет', '16-20 лет', '21-30 лет', 'Более 30 лет']

In [ ]:
doctors

распределение врачей по опыту работы

In [ ]:
plt.figure(figsize=(12, 8))

ax = sns.countplot(data=doctors, y='experience_category', order=category_order, palette='viridis', alpha=0.8)

total = len(doctors)
for p in ax.patches:
    width = p.get_width()
    percentage = f'{100 * width / total:.1f}%'
    x = p.get_x() + width
    y = p.get_y() + p.get_height() / 2
    ax.annotate(f'{int(width)}\n({percentage})', (x, y), ha='left', va='center', fontsize=10)

plt.title('распределение врачей по опыту работы')
plt.xlabel('количество врачей')
plt.ylabel('опыт работы')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

Видим, что больше половины значений пропущены. Восстановить их по другим показателям не представляется возможным. Посмотрим на распределение по квантилям и статистические показатели

In [ ]:
doctors['experience'].describe()

In [ ]:
np.median(doctors[doctors.experience.notna()]['experience'])

In [ ]:
stats.mode(doctors[doctors.experience.notna()]['experience'])

Видим, что по большей части врачи опытные (>10  лет опыта). Медиана практически сходится со средним значением. Значит данные сбалансированы, а выбросы не оказывают на них значимого влияния. Мода - 7 лет.

In [ ]:
sns.kdeplot(doctors['experience'],color='royalblue')
plt.title("Experience KDE")

Видим пик плотности на опыте в 7 лет, затем интенсивность почти монотонно снижается. Возможно как-то связано с началом кризиса COVID-19 в 2020 году. Если исключить всплеск на этом участке, распределение можно было бы считать стремящимся к нормальному.

Ещё один пик - 16-19 лет. Совпадает с рекордным количеством обучающихся в ВУЗах России (2005-2010 годы). Третий пик на показателе 27-28 лет может быть связан с активным созданием новых университетов и развитием высшего образования в конце 1990-ых. [Источник данных.](https://rg.ru/2021/06/15/kolichestvo-rossiian-s-vysshim-obrazovaniem-prevysilo-31-procent.html)

In [ ]:
sns.boxplot(doctors['experience'],color='royalblue')
plt.title("Распределение показателя experience")

Минимум - 0 лет опыта, всё верно. Найдем верхние выбросы (выше конца уса / 3 IQR), если они есть.

In [ ]:
q1 = doctors['experience'].quantile(0.25)
q3 = doctors['experience'].quantile(0.75)
iqr = q3 - q1

upper_whisker = q3 + 1.5 * iqr
upper_3iqr = q3 + 3 * iqr

print(upper_whisker, upper_3iqr, sep='\n')

Выше 3 IQR значений нет

In [ ]:
doctors[doctors['experience'] > upper_3iqr].shape[0]

Выше верхнего уса 18 значений

In [ ]:
doctors[doctors['experience'] > upper_whisker]['experience'].value_counts()

Таким образом, врачи в пожилом возрасте с большим опытом также представлены на площадках

Можем сделать общий вывод, что по данному признаку много пропусков, однако данные распределены относительно симметрично и сбалансированно, выбросы не вносят большого вклада. Присутствует мультимодальность в виде трёх пиков, наибольший из них на показателе в 7 лет. В целом, если пренебречь структурными колебаниями, прослеживается тенденция нормального распределения.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Распределение опыта работы по категориям врачей', fontsize=16, fontweight='bold')

for idx, ax in enumerate(axes.ravel()):
    category = category_order[idx+1]
    data = doctors[doctors.experience_category == category]['experience']

    hist = ax.hist(data, bins=5, alpha=0.7, density=True)

    ax.set_title(f'{category}', fontweight='bold')
    ax.set_xlabel('Experience')
    ax.set_ylabel('Density')
    ax.grid(True, alpha=0.3)
    ax.legend()


plt.tight_layout()
plt.show()

Видим сильную возрастающую тенденцию на категории 0-5 лет (выпуск врачей из ординатуры), небольшой спад после 7 лет (след от структурных изменений) и затем значительный спад после 30 лет (пенсия)

In [ ]:
experienceAndPlatform = doctors.groupby('doctor_source')['experience'].agg(['mean','median','std','min','max'])
experienceAndPlatform

Врачи на сбере в среднем имеют чуть больше опыта

Распределение опыта работы на двух площадках

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

axes[0].hist(doctors[(doctors['doctor_source']=='SberHealth')]['experience'],color='lightgreen')
axes[0].set_title('распределение опыта работы СберЗдоровье')
axes[0].set_xlabel('опыт работы')
axes[0].set_ylabel('количество врачей')
axes[0].grid(alpha=0.3)

axes[1].hist(doctors[doctors['doctor_source']=='ProDoctorov']['experience'],color='cornflowerblue')
axes[1].set_title('распределение опыта работы Продокторов')
axes[1].set_xlabel('опыт работы')
axes[1].set_ylabel('количество врачей')
axes[1].grid(alpha=0.5)

Сбер:
Распределение напоминает нормальное и смещено немного вправо и больше врачей со стажем от 15 до 30 лет.
Пик примерно на 20–25 годах опыта.

Продокторов:
Распределение чуть более равномерное в диапазоне до 30 лет.
Пик ближе к 10–20 годам опыта.

Вывод:
В среднем и по медиане врачи действительно немного опытнее (что подтверждает предыдущая таблица).
**У ProDoctorov больше врачей с меньшим опытом — вероятно,** там чаще регистрируются специалисты в начале карьеры.
Оба распределения широкие

In [ ]:
doctors = doctors.drop(columns=['experience_category'], errors='ignore')

#Rating

In [ ]:
doctors[['rating_sber', 'rating_prod']].describe()

Интерпретация: У Сбера почти все рейтинги сосредоточены в диапазоне 4–5 У Продокторов на контрасте оценки более разбросаны, стандартное отклонение 1,5, встречаются низкие и нулевые значения.

Вывод - СберЗдоровье плохо различает врачей по качеству

Гипотезы:

1) Они могут удалять плохие комментарии (ну плохих оценок просто нет)

2) Могут банить врачей с плохими отзывами, чтобы не портить репутацию платформы тк это сервис по подбору врача, а не просто сервис для отзывов как Продокторов

***При объединении рейтингов важно учитывать, что отзывы на СберЗдоровье могут быть сглажены и несут меньше ценности.

распределение рейтингов Сбера и Продокторов

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1,2,1)
sber_ratings = doctors[doctors['doctor_source'] == 'SberHealth']['rating_sber']
sns.histplot(sber_ratings, bins=20,alpha=0.7,color='lightgreen')
plt.title('распределение рейтингов Сбер')
plt.xlabel('рейтинг')
plt.ylabel('количество врачей')
plt.xlim(0, 5.5)
plt.grid(alpha=0.3)

plt.subplot(1,2,2)
sber_ratings = doctors[doctors['doctor_source'] == 'ProDoctorov']['rating_prod'].dropna()
sns.histplot(sber_ratings, bins=20,alpha=0.7,color='cornflowerblue')
plt.title('распределение рейтингов Продокторов')
plt.xlabel('рейтинг')
plt.ylabel('количество врачей')
plt.xlim(0, 5.5)
plt.grid(alpha=0.3)

In [ ]:
sns.boxplot(x=doctors['rating_sber'],color='cornflowerblue')

In [ ]:
doctors['rating_sber'].describe()

In [ ]:
q1 = doctors['rating_sber'].quantile(0.25)
q3 = doctors['rating_sber'].quantile(0.75)
iqr = q3 - q1

lower_whisker = q1 - 1.5 * iqr # нижний ус
upper_whisker = q3 + 1.5 * iqr # верхний

lower_3iqr = q1 - 3 * iqr
upper_3iqr = q3 + 3 * iqr
print(lower_whisker, upper_whisker, lower_3iqr, sep='\n')

Сравнение рейтингов врачей, которые есть на обеих платформах

In [ ]:
doctors_platforms_with_rating = doctors[
    (doctors['link_sber'].notna()) &
    (doctors['link_prod'].notna()) &
    (doctors['rating_sber'].notna()) &
    (doctors['rating_prod'].notna())].copy()

rating_diff = doctors_platforms_with_rating['rating_sber'] - doctors_platforms_with_rating['rating_prod']
higher_sber = (rating_diff > 0).sum()
higher_prod = (rating_diff < 0).sum()
equal = (rating_diff == 0).sum()

categories = ['выше на SberHealth', 'выше на ProDoctorov', 'равны']
values = [higher_sber, higher_prod, equal]
colors = ['lightgreen','cornflowerblue','bisque']
plt.pie(values, labels=categories, autopct='%1.1f%%',colors=colors)
plt.title('где рейтинг выше')

plt.tight_layout()
plt.show()

Оценки на Сбере в 84 процентах выше, это не норм... Посомтрим на распределение оценок

На сбере узкое распределение значений по оценкам - от 4 до 5. Видимо рейтинги приводят к верхнему диапазону и нормируют плохие оценки к шкале от 4 до 5, чтобы не отпугивать клиентов, ведь это сервис по подборке “хороших врачей"

### Исследуем информацию подробнее

In [ ]:
doctors.info()

### Рассмотрим подробнее review_count_sber

In [ ]:
doctors['review_count_sber'].describe()

У половины врачей количество отзывов равняется 0. При парсинге мы выявили, что рейтинг 0 показывается у врачей, которых нет отзывов (но не всегда). Заменим в этом случае данные на Nan.

In [ ]:
doctors.loc[doctors['rating_sber'] == 0, 'rating_sber'] = np.nan

In [ ]:
plt.figure(figsize=(16, 5))
sns.boxplot(x=doctors['review_count_sber'])

Большинство врачей сосредоточены в очень малых значениях (единицы–десятки отзывов). Есть ооочень много выбросов вплоть до 1000+ отзывов.

У большинства врачей на продокторов совсем мало отзывов (единицы–десятки). Медиана близка к нулю. Выбросов также как и на сберздоровье очень много.

In [ ]:
q1 = doctors['review_count_sber'].quantile(0.25)
q3 = doctors['review_count_sber'].quantile(0.75)
iqr = q3 - q1

upper_whisker = q3 + 1.5 * iqr
upper_3iqr = q3 + 3 * iqr

print(upper_whisker, upper_3iqr, sep='\n')

In [ ]:
doctors['review_count_sber'].quantile(0.95)

У 95% врачей количество отзывов не выше 20, а мы парсили по 20 отзывов со сберздоровья. По сути, мы спарсили почти все отзывы.

In [ ]:
plt.figure(figsize=(16, 5))
sns.boxplot(x=doctors[doctors['review_count_sber'] <= upper_3iqr]['review_count_sber'])

50% врачей имеют около 0–2 отзывов

In [ ]:
ax = sns.histplot(doctors[doctors['review_count_sber'] == 0]['rating_sber'], bins=20,alpha=0.7,color='lightgreen')
ax.set_title('Распределение рейтинга у врачей, у которых нет отзывов')
ax.grid(True, alpha=0.5, linewidth=0.5)

Такое распределение оценки происходит у врачей, у которых нет отзывов

In [ ]:
sns.histplot(doctors[doctors['review_count_sber'] != 0]['rating_sber'], bins=20,alpha=0.7,color='lightgreen')

Такое распределение оценки происходит у врачей, у которых есть отзывы

In [ ]:
sns.histplot(doctors[(doctors['review_count_sber'] == 0) & (doctors['clinic_1_name_sber'].isna())]['rating_sber'], bins=20, alpha=0.7,color='lightgreen')
plt.xlim(0, 5.5)

In [ ]:
doctors[doctors['review_count_sber'] == 0]

Такое распределение оценки происходит у врачей, у которых нет отзывов и нет клиник, в окторых они работают. То есть это неактивные врачи на сберздоровье.

### Рассмотрим подробнее review_count_prod

In [ ]:
doctors['review_count_prod'].describe()

In [ ]:
plt.figure(figsize=(16, 5))
sns.boxplot(x=doctors['review_count_prod'])

У большинства врачей на продокторов совсем мало отзывов. Но медиана равняется двум и она выше, чем у сберздоровья. Выбросов также как и на сберздоровье очень много.

In [ ]:
q1 = doctors['review_count_prod'].quantile(0.25)
q3 = doctors['review_count_prod'].quantile(0.75)
iqr = q3 - q1

upper_whisker = q3 + 1.5 * iqr
upper_3iqr = q3 + 3 * iqr

print(upper_whisker, upper_3iqr, sep='\n')

Но в среднем тут больше отзывов, чем на себрздоровье (максимальное меньше, но минимальное больше).

In [ ]:
plt.figure(figsize=(16, 5))
sns.boxplot(x=doctors[doctors['review_count_prod'] <= upper_3iqr]['review_count_prod'])

50% врачей имеют около 0–2 отзывов. 50% наблюдений лежат в промежутке от 0 до 6.

In [ ]:
doctors[doctors['review_count_prod'] == 0]

In [ ]:
doctors[doctors['review_count_prod'] == 0]['rating_prod'].count()

На продокторов у врачей, у которых нет отзывов и есть рейтинг. Для таких мы указываем NaN.

In [ ]:
sns.histplot(doctors[doctors['review_count_prod'] != 0]['rating_prod'], bins=20,alpha=0.7,color='lightgreen')

Такое распределение оценки происходит у врачей, у которых есть отзывы

In [ ]:
doctors.info()

### Изучаем полученные нами оценки

**rate_our_prod - продокторов**

In [ ]:
doctors['rate_our_prod'].describe()

In [ ]:
sns.histplot(doctors['rate_our_prod'], bins=20,alpha=0.7,color='lightgreen')

Этот график сильно отличается от того, какую информацию мы получили от самого сайта, но это логично, так как мы парсили всего по 3 отзыва, поэтому стоит больше опираться на информацую от продокторов, а не полученную нами.

**rate_our_sber - сберздоровье**

In [ ]:
doctors['rate_our_sber'].describe()

In [ ]:
sns.histplot(doctors['rate_our_sber'], bins=20,alpha=0.7,color='lightgreen')

In [ ]:
bin_edges = np.linspace(0, 5.5, 21)

fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True, sharey=True, constrained_layout=True)

sns.histplot(doctors['rate_our_sber'], bins=bin_edges, color='#b977e6', alpha=0.7, ax=axes[0])
axes[0].set_title('Распределение рейтингов (наш расчёт по СберЗдоровье)')
axes[0].set_xlabel('рейтинг')
axes[0].set_ylabel('количество врачей')
axes[0].grid(alpha=0.3)

sns.histplot(doctors['rating_sber'], bins=bin_edges, color='lightgreen', alpha=0.7, ax=axes[1])
axes[1].set_title('Распределение рейтингов (СберЗдоровье)')
axes[1].set_xlabel('рейтинг')
axes[1].set_ylabel('')
axes[1].grid(alpha=0.3)

plt.show()

Мы видим, что информация, полученнная нами очень сильно разнится с тем, какая информация представлена на сайте, хотя мы спарсили все отзывы для 95% врачей. Соответсвенно, эта полученная нами информация более репрезентативна. Так как оценки идут в среднем не с 3.5 (что очень странно), а разбросаны по всему графику. На сайте сберздоровья пишут, что рейтинг формируется на основании опыта, квалификации, специализации и расписания. Вручную отсмотрев данные мы заметили, что у врачей, количество отзывов на которых равно 0, очень разбросаны оценки непонятно на каком основании. У кого-то 1.8, у кого-то 5, хотя заполненность профилей примерно одинаковая. Также на сайте было написано, что для специалистов проводится тестирование, мы предположили, что оно тоже учитывается в оценке. Тем не менее, нами был сделан вывод, что многие оценки на сберздоровье поставлены рандомно.

### Общий столбец рейтинга

In [ ]:
doctors.loc[doctors['rating_sber'] < 3, 'rating_sber'] = 3


border = 3
doctors['rating_sber_norm'] = (
    (doctors['rating_sber'] - border) / (5 - border) * (5 - 1.5) + 1.5
)

doctors['rating_sber_norm'].describe()

In [ ]:
bin_edges = np.linspace(0, 5.5, 21)

fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True, sharey=True, constrained_layout=True)

sns.histplot(doctors['rate_our_sber'], bins=bin_edges, color='#b977e6', alpha=0.7, ax=axes[0])
axes[0].set_title('Распределение рейтингов (наш расчёт)')
axes[0].set_xlabel('рейтинг')
axes[0].set_ylabel('количество врачей')
axes[0].grid(alpha=0.3)

sns.histplot(doctors['rating_sber_norm'], bins=bin_edges, color='lightgreen', alpha=0.7, ax=axes[1])
axes[1].set_title('Распределение рейтингов после нормализации')
axes[1].set_xlabel('рейтинг')
axes[1].set_ylabel('')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1,2,1)
sns.histplot(doctors['rating_sber_norm'], bins=20,alpha=0.7,color='lightgreen')
plt.title('нормированное распределение рейтингов Сбер')
plt.xlabel('рейтинг')
plt.ylabel('количество врачей')
plt.xlim(0, 5.5)
plt.grid(alpha=0.3)

plt.subplot(1,2,2)
sns.histplot(doctors['rating_prod'], bins=20,alpha=0.7,color='cornflowerblue')
plt.title('распределение рейтингов Продокторов')
plt.xlabel('рейтинг')
plt.ylabel('количество врачей')
plt.xlim(0, 5.5)
plt.grid(alpha=0.3)

In [ ]:
q99 = doctors['review_count_sber'].quantile(0.99)
q99

In [ ]:
weights = {
    'rating_prod':   0.50,
    'rate_our_sber': 0.25,
    'rating_sber_norm':   0.15,
    'rate_our_prod': 0.10,
}

count_columns = {
    'rating_prod':   'review_count_prod',
    'rate_our_prod': 'comment_prod',
    'rating_sber_norm':   'review_count_sber',
    'rate_our_sber': 'comment_sber',
}

In [ ]:
# берем верхнее значение 100, так как это близко к 99 квантилю и более менее общепринятое значение
max_reviews = 100
min_reliability_if_no_n = 0.5 # что делать, если рейтинг есть, а n нет/0

def calc_rating(row):
    """
    Общий рейтинг (0–5) как взвешенное среднее.
    Вес = base_weight * log1p(min(n, max_reviews)),
    Если n>0, то берём минимальную надёжность (1.0).
    """
    total_weighted_score = 0.0 # сумма (вес * рейтинг)
    total_effective_weight = 0.0 # сумма весов
    any_rating_present = False # был ли хоть один рейтинг вообще

    for rating_col, base_weight in weights.items(): # обходим все источники рейтингов и их базовые веса
        rating_value = row.get(rating_col) # берём значение рейтинга из текущей строки по колонке rating_col
        if pd.notna(rating_value):
            any_rating_present = True # есть хоть какой-то рейтинг

            reviews_count_col = count_columns[rating_col]
            reviews_count = row.get(reviews_count_col)

            if pd.notna(reviews_count) and reviews_count > 0: # узнаём в какой колонке лежит число отзывов для этого источника
                reliability = np.log1p(min(reviews_count, max_reviews)) # если n известно и > 0 то считаем надёжность источника как log(1+n) с ограничением max_reviews. лог используем чтобы был рост с n, но с убывающей отдачей: 10-20 отзывов важнее, чем 1010-1020
            else:
                reliability = min_reliability_if_no_n # если n нет или 0, всё равно учитываем рейтинг, но даём минимальную надёжность

            effective_weight = base_weight * reliability # вес источника = базовый вес * надёжность по n
            total_weighted_score  += effective_weight * rating_value
            total_effective_weight += effective_weight # накапливаем вклад источника в числитель и сам вес в знаменатель

    # NaN только если вообще нет ни одного рейтинга
    if not any_rating_present:
        return np.nan

    return total_weighted_score / total_effective_weight

In [ ]:
doctors['rating'] = doctors.apply(calc_rating, axis=1)
doctors

In [ ]:
ax = sns.histplot(doctors['rating'], bins=20,alpha=0.7, color='#b977e6', kde=True)
ax.lines[-1].set_color('black')

plt.title('Распределение общего рейтинга')
plt.xlabel('рейтинг')
plt.ylabel('количество врачей')

#Цена (`price`)

##основные характеристики

In [ ]:
doctors[['price_sber', 'price_prod']].describe()

Основные характеристики цен на консультации врача на СберЗдоровье чуть выше, чем на ПроДокторов. Но не сильно, все равно виден схожий “массовый” уровень цен и  наличие выбросов

##выбросы

### sber

In [ ]:
Q1 = doctors['price_sber'].quantile(0.25)
Q3 = doctors['price_sber'].quantile(0.75)
IQR = Q3 - Q1
lower_whisker = Q1 - 1.5 * IQR
upper_whisker = Q3 + 1.5 * IQR
print(lower_whisker,upper_whisker)

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors, x='price_sber',color = 'lightgreen')

рассмотрим без выбросов выше 3IQR

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors[doctors['price_sber'] < 3*IQR + Q3], x='price_sber',color='lightgreen')

Большинство цен лежат примерно в диапазоне 3000–5500

### prodoctorov

In [ ]:
Q1 = doctors['price_prod'].quantile(0.25)
Q3 = doctors['price_prod'].quantile(0.75)
IQR = Q3 - Q1
lower_whisker = Q1 - 1.5 * IQR
upper_whisker = Q3 + 1.5 * IQR
print(lower_whisker,upper_whisker)

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors, x='price_prod', color='cornflowerblue')

аналогично рассмотрим значение < 3IQR

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors[doctors['price_prod'] < 3*IQR + Q3], x='price_prod',color='cornflowerblue')

Основной диапазон цен от 2800 до 5400 ₽, почти идентичен диапазону на Сбере 3000–5500.
Минимальные цены стартуют с 500, что говорит о наличии более дешёвых консультаций — возможно, молодых специалистов или скидок.

##cравнение цен для одних и тех же врачей на разных платформах

In [ ]:
plt.figure(figsize=(12, 6))

doctors_platforms_with_price = doctors[
    (doctors['link_sber'].notna()) &
    (doctors['link_prod'].notna()) &
    (doctors['price_sber'].notna()) &
    (doctors['price_prod'].notna())]

plt.scatter(doctors_platforms_with_price['price_sber'], doctors_platforms_with_price ['price_prod'], alpha=0.6, color='cornflowerblue', s=60)
plt.title('cравнение цен для одних и тех же врачей на разных платформах')
plt.xlabel('цена SberHealth')
plt.ylabel('цена ProDoctorov')
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

Есть положительная зависимость, врачи сохраняют примерно одинаковый ценовой уровень на обеих площадках.

Но! Рассеивание большое
Видно множество точек, у которых цена на одной платформе выше, чем на другой. Часть врачей дифференцирует цены в зависимости от платформы — вероятно, по аудитории

In [ ]:
corr = doctors_platforms_with_price['price_sber'].corr(doctors_platforms_with_price ['price_prod'])
corr

цены и правда +- такие же, если, они, конечно, есть

##добавление столбца price

*у нас высокая корреляция у двух столбцов(никакой не является целевой переменной), тогда сделаем общий столбец

price = (price_sber+price_prod)/2*

In [ ]:
doctors['price'] = (doctors['price_prod']+doctors['price_sber'])/2

In [ ]:
doctors['price'].describe()

In [ ]:
doctors

In [ ]:
Q1 = doctors['price'].quantile(0.25)
Q3 = doctors['price'].quantile(0.75)
IQR = Q3 - Q1
lower_whisker = Q1 - 1.5 * IQR
upper_whisker = Q3 + 1.5 * IQR
print(lower_whisker,upper_whisker)

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors, x='price',color = 'bisque')

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors[doctors['price'] < 3*IQR], x='price',color = 'bisque')

In [ ]:
plt.figure(figsize=(15, 5))

sns.histplot(doctors[(doctors['price'].notna()) & (doctors['price'] < 3*IQR)]['price'],bins=50, color = 'bisque',edgecolor='salmon')
plt.title('Распределение цен')
plt.xlabel('Цена')
plt.ylabel('Количество врачей')
plt.grid(alpha=0.3,axis='y')

##распределения цен на платформах

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
sns.histplot(doctors[(doctors['price_sber'].notna()) & ((doctors['price_sber'] < 3*IQR))]['price_sber'],
             bins=50, color = 'lightgreen',edgecolor='forestgreen')
plt.title('Распределение цен на SberHealth')
plt.xlabel('Цена')
plt.ylabel('Количество врачей')
plt.grid(alpha=0.3,axis='y')

plt.subplot(1, 2, 2)
sns.histplot(doctors[(doctors['price_prod'].notna()) & (doctors['price_prod'] < 3*IQR)]['price_prod'],
             bins=50,color = 'cornflowerblue',edgecolor='royalblue')
plt.title('Распределение цен на ProDoctorov')
plt.xlabel('Цена')
plt.ylabel('Количество врачей')
plt.grid(alpha=0.3,axis='y')

plt.tight_layout()
plt.show()

##ценовые категории для sber, prodoctorov, AvgPrice

In [ ]:
bins = [0, 1500, 3000, 5000, 7500, 10000, 15000, 25000, float('inf')]
labels = ['до 1.5k', '1.5k-3k', '3k-5k', '5k-7.5k', '7.5k-10k', '10k-15k', '15k-25k', '25k+']

doctors['price_category_sber'] = pd.cut(doctors['price_sber'], bins=bins, labels=labels)
doctors['price_category_prod'] = pd.cut(doctors['price_prod'], bins=bins, labels=labels)
doctors['price_category_avg'] = pd.cut(doctors['price'], bins=bins, labels=labels)

plt.figure(figsize=(12, 6))

sber_counts = doctors['price_category_sber'].value_counts().sort_index()
prod_counts = doctors['price_category_prod'].value_counts().sort_index()
price_counts = doctors['price_category_avg'].value_counts().sort_index()

x = np.arange(len(labels))
width = 0.35

bars1 = plt.bar(x - width/2, sber_counts, width, label='СберЗдоровье', alpha=0.7, color = 'lightgreen')
bars2 = plt.bar(x + width/2, prod_counts, width, label='ProDoctorov', alpha=0.7, color = 'cornflowerblue')
bars3 = plt.bar(x, price_counts, width, label='AvgPrice', color = 'slategrey')

for i, count in enumerate(sber_counts):
    plt.text(i-0.18, count + 20, str(count), ha='center')
for i, count in enumerate(prod_counts):
    plt.text(i+0.18, count + 20, str(count), ha='center')
for i, count in enumerate(price_counts):
    plt.text(i, count + 20, str(count), ha='center')

plt.xlabel('Ценовые категории')
plt.ylabel('Количество врачей')
plt.title('Врачи по ценовым категориям')
plt.xticks(x, labels)
plt.grid(alpha=0.3,axis='y')
plt.legend()
plt.tight_layout()
plt.show()


Основная масса рынка до 7.5k.

Больше 70 % врачей находятся в диапазоне 1.5k–5k

Категория 3k–5k ₽ — самая популярная:

Обе платформы наиболее насыщены врачами со средней ценой — конкуренция и выбор максимальны именно здесь.

Низкобюджетный сегмент почти отсутствует, что говорит о минимальном демпинге на рынке.

Дорогие приёмы редки — премиальный рынок ограничен, но вероятно ориентирован на центр города и узкие специальности

## зависимость цены от стажа по платформам

In [ ]:
doctors[(doctors['doctor_source']=='SberHealth')][['experience','price_sber']].dropna().shape
doctors[(doctors['doctor_source']=='ProDoctorov')][['experience','price_prod']].dropna().shape


у нас в датасете 71 врач у которых одновременно и опыт и цена указаны..

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(doctors[doctors['doctor_source']=='SberHealth']['experience'],
            doctors[doctors['doctor_source']=='SberHealth']['price_sber'], alpha=0.6, color='lightgreen', s=60, label='SberHealth')

plt.scatter(doctors[doctors['doctor_source']=='ProDoctorov']['experience'],
            doctors[doctors['doctor_source']=='ProDoctorov']['price_prod'], alpha=0.6, color='cornflowerblue', s=60, label='ProDoctorov')

plt.title('зависимость цены от стажа по платформам')
plt.xlabel('опыт работы')
plt.ylabel('цена')
plt.grid(alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
corr_sber = doctors[doctors['doctor_source']=='SberHealth']['experience'].corr(doctors[doctors['doctor_source']=='SberHealth']['price_sber'])
corr_prod = doctors[doctors['doctor_source']=='ProDoctorov']['experience'].corr(doctors[doctors['doctor_source']=='ProDoctorov']['price_prod'])
print(corr_sber,corr_prod)

## сравнение средних цен по специальностям на разных платформах

In [ ]:
specialties = ['гинеколог', 'кардиолог', 'невролог', 'онколог', 'терапевт', 'эндокринолог']
avg_prices = pd.DataFrame(index=specialties, columns=['price_sber', 'price_prod'])

for specialty in specialties:
    specialty_doctors = doctors[doctors['speciality'].str.contains(specialty,na=False)]

    avg_sber = specialty_doctors['price_sber'].mean()
    avg_prices.loc[specialty, 'price_sber'] = avg_sber

    avg_prod = specialty_doctors['price_prod'].mean()
    avg_prices.loc[specialty, 'price_prod'] = avg_prod
avg_prices = avg_prices.astype(float)


plt.figure(figsize=(8, 6))
x = np.arange(len(specialties))
width = 0.35
plt.bar(x - width/2, avg_prices['price_sber'], width, label='SberHealth', alpha=0.8, color='lightgreen')
plt.bar(x + width/2, avg_prices['price_prod'], width, label='ProDoctorov', alpha=0.8, color='cornflowerblue')

plt.xlabel('Специальность')
plt.ylabel('Средняя цена')
plt.title('Сравнение средних цен по специальностям на разных платформах')
plt.xticks(x, specialties)
plt.legend()
plt.grid(axis='y', alpha=0.3)

for i in range(len(specialties)):
    sber_price = avg_prices['price_sber'].iloc[i]
    prod_price = avg_prices['price_prod'].iloc[i]
    plt.text(i - width/2, sber_price + 30, f'{sber_price:.0f}', ha='center', va='bottom', fontsize=9)
    plt.text(i + width/2, prod_price + 30, f'{prod_price:.0f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

СберЗдоровье дороже по всем специальностям


Онкологи — самые дорогие врачи на обеих платформах
Средняя цена около 6 000 ₽.
Редкая и высококвалифицированная категория, подтверждает премиум-сегмент.


Терапевты — самые доступные
На ПроДокторов — около 4 200 ₽, что на 30% ниже, чем у онкологов.


Разница между платформами стабильна по всем направлениям

## посмотрим на разницы в ценах:

In [ ]:
price_diff = doctors_platforms_with_price['price_sber'] - doctors_platforms_with_price['price_prod']
price_diff.describe()

In [ ]:
doctors_with_price_diff = doctors.copy()
doctors_with_price_diff['price_diff'] = abs(doctors_platforms_with_price['price_sber'] - doctors_platforms_with_price['price_prod']) # по модулю
doctors_with_price_diff['price_diff'].describe()

среднее очень отличается от медианы

In [ ]:
plt.figure(figsize=(16, 2))
sns.boxplot(data=doctors_with_price_diff, x='price_diff',color='royalblue')

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=doctors_with_price_diff[doctors_with_price_diff['price_diff'] > 0], x='price_diff')

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=doctors_with_price_diff[doctors_with_price_diff['price_diff'] > 0], x='price_diff',color='royalblue')
plt.xlim(0, doctors_with_price_diff['price_diff'].quantile(0.97))

Пик в диапазоне 0–1000 ₽ — значит, у большинства врачей расхождение между Сбером и ПроДоктором минимальное, хвост доходит до 5–6 тыс., что всё ещё заметная разница. Распределение остается скошенным

Очень много выбросов. Оценим по верхнему усу и 3 IQR

In [ ]:
q1 = doctors_with_price_diff['price_diff'].quantile(0.25)
q3 = doctors_with_price_diff['price_diff'].quantile(0.75)
iqr = q3 - q1

upper_whisker = q3 + 1.5 * iqr
upper_3iqr = q3 + 3 * iqr

print(upper_whisker, upper_3iqr, sep='\n')

In [ ]:
doctors_with_price_diff[doctors_with_price_diff['price_diff'] > upper_whisker].shape[0]

In [ ]:
doctors_with_price_diff[doctors_with_price_diff['price_diff'] > upper_3iqr].shape[0]

Посмотрим на те, где выбросы больше 3 IQR

In [ ]:
doctors_with_price_diff[doctors_with_price_diff['price_diff'] > upper_3iqr][['price_sber', 'price_prod', 'price_diff']]

In [ ]:
doctors_with_price_diff[doctors_with_price_diff['price_diff'] > upper_3iqr][['price_sber', 'price_prod', 'price_diff']].describe()

В этих случаях в осеовном выше на сбере

In [ ]:
doctors_with_price_diff[doctors_with_price_diff['price_diff'] > q3 + 9*iqr][['price_sber', 'price_prod', 'price_diff']]

Для самых высоких значений разниц цен - очень высокие цены на Продокторов, в 13 из 15 случаев (отклонения выше 9 IQR)

## где цена выше для одних и тех же враче

In [ ]:
higher_sber = (price_diff > 0).sum()
higher_prod = (price_diff < 0).sum()
equal = (price_diff == 0).sum()

categories = ['выше на SberHealth', 'выше на ProDoctorov', 'равны']
values = [higher_sber, higher_prod, equal]
colors = ['lightgreen','cornflowerblue','bisque']
plt.pie(values, labels=categories, autopct='%1.1f%%',colors=colors)
plt.title('где цена выше для одних и тех же врачей')

plt.tight_layout()
plt.show()

In [ ]:
corr = doctors_platforms_with_price['price_sber'].corr(doctors_platforms_with_price['price_prod'])
corr

#Специальности (`'speciality'`)

In [ ]:
doctors.columns

In [ ]:
def speciality_to_string(spec_list):
    if pd.isna(spec_list):
        return None
    if isinstance(spec_list, str):
        try:
            spec_list = eval(spec_list)
        except:
            return spec_list
    if isinstance(spec_list, list):
        return ', '.join(spec_list)
    return str(spec_list)
doctors['speciality'] = doctors['speciality'].apply(speciality_to_string)

In [ ]:
doctors['speciality']

In [ ]:
doctors['speciality'].unique()

In [ ]:
all_individual_specs = []
for spec_str in doctors['speciality'].dropna():
    specs = spec_str.split(', ')
    all_individual_specs.extend(specs)

individual_spec_counts = pd.Series(all_individual_specs).value_counts()

In [ ]:
individual_spec_counts.sample(10)

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
top_25_specs = individual_spec_counts.head(25)
top_25_specs.plot(kind='barh', alpha=0.7,color='royalblue')
plt.title('частота топ-25 специальностей')
plt.xlabel('количество упоминаний')
plt.grid(axis='x', alpha=0.3)

for i, count in enumerate(top_25_specs.values):
    plt.text(count, i, str(count), va='center', fontsize=7)

plt.tight_layout()
plt.show()

## популярные специальности среди терапевтов

In [ ]:
terapevts = doctors[doctors['speciality'].str.contains('терапевт', case=False, na=False)]

terapevt_individual_specs = []
for spec_str in terapevts['speciality'].dropna():
    specs = spec_str.split(', ')
    terapevt_individual_specs.extend(specs)

terapevt_spec_counts = pd.Series(terapevt_individual_specs).value_counts()

In [ ]:
terapevt_spec_counts.head(20)

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
top_25_specs = terapevt_spec_counts[1:26]
top_25_specs.plot(kind='barh', alpha=0.7,color='royalblue')
plt.title('частота топ-25 специальностей у терапевтов (помимо "терапевт")')
plt.xlabel('количество упоминаний')
plt.grid(axis='x', alpha=0.3)

for i, count in enumerate(top_25_specs.values):
    plt.text(count, i, str(count), va='center', fontsize=7)

plt.tight_layout()
plt.show()

#Детские и взрослые врачи (`kids/adults`)

In [ ]:
plt.figure(figsize=(9, 6))

categories_count = {
    'only взрослые': doctors['is_adults'].sum(),
    'only дети': doctors['is_kids'].sum(),
    'и дети и взрослые': ((doctors['is_adults'] == True) & (doctors['is_kids'] == True)).sum(),
    'неть ничего': ((doctors['is_adults'] == False) & (doctors['is_kids'] == False)).sum()}

colors = sns.color_palette("viridis", len(categories_count))
plt.bar(categories_count.keys(), categories_count.values(),color=colors)
plt.title('распределение по категориям пациентов')
plt.ylabel('количество врачей')
plt.grid(axis='y', alpha=0.3)

for i, count in enumerate(categories_count.values()):
    plt.text(i, count, str(count), ha='center', va='bottom')

### Разобьем специальности на разные строки

In [ ]:
def to_list(text):
    if pd.isna(text):
        return []
    return [t.strip() for t in str(text).split(',') if t.strip()]

In [ ]:
new_df = doctors.copy()
new_df['speciality'] = new_df['speciality'].apply(to_list)

### Исследуем ко-встречаемость специальностей

In [ ]:
flat = [sp for L in new_df["speciality"] for sp in L] # берём каждую строку (L — это список спец. врача) и из каждого такого списка берём каждую спец. (sp). В итоге flat — это один длинный список всех спец. по всем врачам
vc = pd.Series(flat).value_counts() # считаем, сколько раз каждая спец. встречается. получаем частоты по всем спец.

TOPN = 25 # берем только 25 самых популярных специальностей
topN = vc.head(TOPN).index # получаем эти специальности

# специальный счётчик из модуля collections. он ведёт частоты объектов как словарь: ключ -> сколько раз встретился
pair_counter = Counter() # сколько раз встретилась каждая спец.

for L in new_df["speciality"]:
    S = [sp for sp in L if sp in topN] # оставляем только те спец. этого врача, которые входят в наш топ-25
    if not S:
        continue
    S = sorted(set(S)) # set(S), чтобы внутри одной строки не учитьывать два раза одну и ту же специальность. сортируем для того, чтобы был единый лексикографический порядок
    for a, b in combinations(S, 2): # берем все комбинации по 2
        pair_counter[(a,b)] += 1
        pair_counter[(b,a)] += 1 # делаем матрицу симметричной

C = pd.DataFrame(0, index=topN, columns=topN, dtype=int) # создаём квадратную таблицу, всю забитую нулями
for (a,b), c in pair_counter.items():
    C.loc[a,b] = c # заполняем ячейки числами«сколько раз пара (a,b) встретилась у врачей
np.fill_diagonal(C.values, 0) # чтобы не сбивать с толку на диагонали поставим значение 0

masked = np.ma.masked_where(C.to_numpy() == 0, C.to_numpy()) # всё, где матрица C равна нулю, помечаем как маску (невидимые значения)
cmap = plt.cm.viridis.copy() # берем копию колормэпа viridis, чтобы можно было менять его свойства не трогая глобальный)
cmap.set_bad('#f2f2f2') # так рисуем нули

fig, ax = plt.subplots(figsize=(11, 9))
im = ax.imshow(masked, cmap=cmap, aspect="auto", vmin=1, vmax=C.values.max()) # разрешаем оси растягиваться по размеру фигуры, нижняя граница шкалы = 1, чтобы 0 точно не окрашивался как данные, верхняя граница шкалы = максимум в C
ax.set_xticks(range(C.shape[1])); ax.set_yticks(range(C.shape[0])) # ставим подписи на каждую колонку и строку матрицы
ax.set_xticklabels(C.columns, rotation=90); ax.set_yticklabels(C.index) # подписываем их значения
ax.set_title(f"Ко-встречаемость специальностей")
plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

In [ ]:
new_df = new_df.explode('speciality')
new_df

Удалим словой 'детский' из специальностей и пометим is_kids = True, если встретили такой случай. Также флаг помечаем 1, если встретили слово 'педиатр', но не удаляем его

In [ ]:
KIDS_ADJ_RE = re.compile(r"\b(детск\w*|педиатрическ\w*)\b", flags=re.IGNORECASE) # компилируем объект регулярного выражения для последующего использования, игнорируем регистр
PEDIATR_ONLY_RE = re.compile(r"\bпедиатр\b", flags=re.IGNORECASE)

def clean_kids_flag(spec: str):
    if not isinstance(spec, str): # если каким-то образом в функцию попало не строковое значение (например NaN), просто возвращаем как есть и is_kids=False
        return spec, False

    s = spec.strip()
    has_kids_adj = bool(KIDS_ADJ_RE.search(s)) # если нашли нашу регулярку, то флаг принимает значение True
    has_pediatr = bool(PEDIATR_ONLY_RE.search(s)) # если нашли нашу регулярку, то флаг принимает значение True

    # вырезаем только прилагательные-маркеры, просто педиатров не трогаем
    if has_kids_adj:
        s = KIDS_ADJ_RE.sub("", s) # если нашли прилагательные-маркеры (из первого флага), вырезаем их из названия специальности

    # чистим дефисы/тире и лишние пробелы, которые могли остаться, убираем висячие дефисы вокруг мест вырезки
    s = re.sub(r"\s{2,}", " ", s).strip(" ,.;-–—")

    is_kids = has_kids_adj or has_pediatr

    return s, is_kids

new_df[["speciality", "is_kids"]] = (new_df["speciality"].apply(clean_kids_flag).apply(pd.Series))
new_df["is_kids"] = new_df["is_kids"].astype(bool)

In [ ]:
new_df

## Топ-20 врачей (учитываются только врачи для взрослых)

In [ ]:
adults_df = new_df[new_df['is_adults'] == True]

In [ ]:
counts = (adults_df['speciality'].dropna().value_counts().head(20)) # считаем, сколько раз встречается каждая специальность, берём топ-20 самых частых

counts = counts.sort_values(ascending=True) # сортируем по возрастанию
fig, ax = plt.subplots(figsize=(14, 6))
ax.barh(counts.index, counts.values, color= '#5e3c6e')

for y, v in enumerate(counts.values): # проходим по всем столбцам. y - номер строки (позиция по вертикали), v — значение (длина столбика)
    ax.text(v + max(counts.values)*0.01, y, str(v), va="center") # рисуем подпись с числом справа от столбика. v + max(counts.values)*0.01 — текст чуть правее конца столбика. y — вертикальная позиция подписи ровно напротив столбика. va="center" — выравнивание текста по вертикали по центру столбика

ax.set_xlabel('Количество')
ax.set_ylabel('Специальность')
ax.set_title('Топ-20 специальностей для взрослых людей')

plt.tight_layout()
plt.show()

## Топ-20 врачей (учитываются толькое детские)

In [ ]:
kids_df = new_df[new_df['is_kids'] == True]

In [ ]:
counts = (kids_df['speciality'].dropna().value_counts().head(20)) # считаем, сколько раз встречается каждая специальность, берём топ-20 самых частых

counts = counts.sort_values(ascending=True) # сортируем по возрастанию
fig, ax = plt.subplots(figsize=(14, 6))
ax.barh(counts.index, counts.values, color= '#5e3c6e')

for y, v in enumerate(counts.values): # проходим по всем столбцам. y - номер строки (позиция по вертикали), v — значение (длина столбика)
    ax.text(v + max(counts.values)*0.01, y, str(v), va="center") # рисуем подпись с числом справа от столбика. v + max(counts.values)*0.01 — текст чуть правее конца столбика. y — вертикальная позиция подписи ровно напротив столбика. va="center" — выравнивание текста по вертикали по центру столбика

ax.set_xlabel('Количество')
ax.set_ylabel('Специальность')
ax.set_title('Топ-20 детских специальностей')

plt.tight_layout()
plt.show()

## Топ-30 редких специальностей среди взрослых

In [ ]:
freq = adults_df['speciality'].value_counts() # считаем частоту встречаний каждого значения
freq_sorted = freq.sort_values(ascending=False) # сортируем по убыванию
rare = freq[freq <= 10].sort_values(ascending=True) # считаем редкими тех, кто встречается не более 10 раз

In [ ]:
plt.figure(figsize=(15, 8))
plt.barh(rare.index, rare.values, color= '#5e3c6e')

for y, v in enumerate(rare.values):
    plt.text(v + (rare.values.max()*0.02 if rare.values.max()>0 else 0.1), y, str(v), va="center")

plt.xlabel("Количество")
plt.ylabel("Специальность")
plt.title("Топ-30 редких специальностей")

plt.tight_layout()
plt.show()

## Топ-30 редких специальностей среди детей

In [ ]:
freq = kids_df['speciality'].value_counts() # считаем частоту встречаний каждого значения
freq_sorted = freq.sort_values(ascending=False) # сортируем по убыванию
rare = freq[freq <= 10].sort_values(ascending=True) # считаем редкими тех, кто встречается не более 10 раз

In [ ]:
plt.figure(figsize=(15, 8))
plt.barh(rare.index, rare.values, color= '#5e3c6e')

for y, v in enumerate(rare.values):
    plt.text(v + (rare.values.max()*0.02 if rare.values.max()>0 else 0.1), y, str(v), va="center")

plt.xlabel("Количество")
plt.ylabel("Специальность")
plt.title("Топ-30 редких специальностей среди детей")

plt.tight_layout()
plt.show()

## Доли детских врачей

In [ ]:
ct = pd.crosstab(new_df["speciality"], new_df["is_kids"]) # строим перекрёстную таблицу
colors = ["#5e3c6e", "#f59e42"]

share = ct.div(ct.sum(1), axis=0).loc[ct.sum(1).nlargest(20).index] # делим каждую строку ct на её собственную сумму, берём индексы топ-20 самых многочисленных специальностей по общему числу врачей
ax = share.plot(kind="barh", stacked=True, figsize=(10, 8), color=colors, edgecolor="white")

leg = ax.legend(title="is_kids", bbox_to_anchor=(1.02, 0.5), frameon=False)

### Рассмотрим средние цены на специальности в зависимости от того, каких пациентов принимает врач

In [ ]:
def label_row(r):
    if r['is_kids']==1 and r['is_adults']==1:
        return 'Оба'
    elif r['is_kids']==1:
        return 'Детские'
    elif r['is_adults']==1:
        return 'Взрослые'
groups_order = ['Детские','Взрослые','Оба']
colors = {
    'Детские': '#c76bb6',
    'Взрослые': '#6ba4c7',
    'Оба': '#8ce880',
}

new_df['group'] = new_df.apply(label_row, axis=1)

# возьмём топ-10 популярных специальностей по числу записей)
top_specs = new_df['speciality'].value_counts().head(10).index
doctors_top = new_df[new_df['speciality'].isin(top_specs)].copy()

# порядок специальностей - по средней цене для взрослых
order = (doctors_top[doctors_top['group']=='Взрослые'].groupby('speciality')['price'].mean().sort_values(ascending=False))
order = order.index
doctors_top['speciality'] = pd.Categorical(doctors_top['speciality'], categories=order, ordered=True)

med = (doctors_top
       .groupby(['speciality','group'])['price']
       .mean()
       .unstack() # столбцы = группы
       .reindex(columns=groups_order)) # фиксируем порядок столбцов

x = np.arange(len(med.index))
w = 0.26

plt.figure(figsize=(12,5))
for i, g in enumerate(groups_order):
    if g in med.columns:
        plt.bar(x + (i-1)*w, med[g].values, width=w, label=g, color=colors.get(g, None))
        '''
        Идем по группам в нужном порядке. enumerate даёт индекс i (0,1,2). смещаем столбики каждой группы относительно центров x:
        при i=0 позиция x - w (левее), при i=1 позиция x + 0*w (по центру), при i=2 позиция x + w (правее)
        '''

plt.xticks(x, med.index, rotation=30, ha='right')
plt.ylabel('Средняя цена')
plt.title('Средние цены по специальностям')
plt.legend(frameon=False)
plt.tight_layout()
plt.show()

Видим, что у онкологов принимающих детей цена сильно выше всех остальных. Далее цены для детей и взрослых примерно одинаковые

### Сравнение рейтинга у детских и взрослых врачей

In [ ]:
# возьмём топ-10 популярных специальностей по числу записей)
top_specs = new_df['speciality'].value_counts().head(10).index
doctors_top = new_df[new_df['speciality'].isin(top_specs)].copy()

# порядок специальностей - по среднему рейтингу для взрослых
order = (doctors_top[doctors_top['group']=='Взрослые'].groupby('speciality')['rating'].mean().sort_values(ascending=False))
order = order.index
doctors_top['speciality'] = pd.Categorical(doctors_top['speciality'], categories=order, ordered=True)

med = (doctors_top
       .groupby(['speciality','group'])['rating']
       .mean()
       .unstack() # столбцы = группы
       .reindex(columns=groups_order)) # фиксируем порядок столбцов

x = np.arange(len(med.index))
w = 0.26

plt.figure(figsize=(12,5))
for i, g in enumerate(groups_order):
    if g in med.columns:
        plt.bar(x + (i-1)*w, med[g].values, width=w, label=g, color=colors.get(g, None))
        '''
        Идем по группам в нужном порядке. enumerate даёт индекс i (0,1,2). смещаем столбики каждой группы относительно центров x:
        при i=0 позиция x - w (левее), при i=1 позиция x + 0*w (по центру), при i=2 позиция x + w (правее)
        '''

plt.xticks(x, med.index, rotation=30, ha='right')
plt.ylabel('Средний рейтинг')
plt.title('Средние рейтинги по специальностям')
plt.legend(title=None, frameon=False, loc='upper left', bbox_to_anchor=(1.02, 1))
plt.tight_layout()
plt.tight_layout()
plt.show()

#Метро (`metro`)

In [ ]:
plt.figure(figsize=(12,6))

all_metro_stations = []
metro_columns = [col for col in doctors.columns if 'metro' in col]
for col in metro_columns:
    metro_data = doctors[col].dropna()
    all_metro_stations.extend(metro_data)

metro_counts = pd.Series(all_metro_stations).value_counts()
top_20_metro = metro_counts.head(20)
top_20_metro.plot(kind='barh', color='royalblue', alpha=0.7)
plt.title('top-20 metro stations')
plt.xlabel('count clinics')
plt.grid(axis='x', alpha=0.3)

for i, count in enumerate(top_20_metro.values):
    plt.text(count, i, str(count), va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
metro_counts

# Число клиник (clinics_count)

In [ ]:
int(doctors['clinics_count_sber'].max())

In [ ]:
doctors['clinics_count_sber'].value_counts()

In [ ]:
doctors['clinics_count_prod'].value_counts()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
ax1, ax2 = axes

sns.histplot(data=doctors, x='clinics_count_sber', bins=int(doctors['clinics_count_sber'].max()), ax=ax1,color = 'lightgreen')
ax1.set_title('Распределение количества клиник (Сбер)')
ax1.set_xlabel('Количество клиник')
ax1.set_ylabel('Количество врачей')
ax1.set_ylim(0, 27559+1000)

sns.histplot(data=doctors, x='clinics_count_prod', bins=int(doctors['clinics_count_prod'].max()), ax=ax2,color = 'cornflowerblue')
ax2.set_title('Распределение количества клиник (Продокторов)')
ax2.set_xlabel('Количество клиник')
ax2.set_ylabel('Количество врачей')
ax2.set_ylim(0, 27559+1000)

plt.tight_layout()
plt.show()

In [ ]:
doctors['clinics_count_prod'].value_counts()

На Продокторов чаще есть информация о клиниках

#Основные выводы по унивариативному анализу и анализу в целом на текущий момент



1) **Качество и структура данных**

В выборке около 40 000 врачей. Заполненность данных очень сильно варьируется: цена и опыт очень много где не заполнены. Стаж можно не указывать, а цена скрывается если врач не принимает.Тем не менее мы можем сравнить ключевые показатели — стоимость, рейтинг и распространённость специальностей.

**Вывод:** данные репрезентативны для анализа рыночных тенденций и ценовых различий между платформами, несмотря на естественные пропуски.
Это кстати как раз подтверждает актуальность идеи нашего проекта: пользователи не могут получить единый, прозрачный источник. Даже при ограниченных данных видно, что цены, рейтинги и характеристики врачей несогласованы

2) **Распределение цен**

Распределение конечно асимметричное, с длинным правым хвостом.
Основная масса врачей принимает в диапазоне 2 500–4 000.
Выбросы встречаются вплоть до 150 000 за сессию))), что мы видим у узких премиум специалистов.
Разброс цен внутри одной специальности остаётся значительным

**Вывод:** медианный приём в Москве стоит около 3 000, но рынок сильно сегментирован по цене и специализации.

3) **Сравнение платформ**

Для одних и тех же специалистов:
45% цен совпадают,
40% выше на СберЗдоровье,

**Вывод:** ценовая политика платформ в целом схожа, но СберЗдоровье имеет тенденцию к чуть более высоким ценам


#Мультивариантивный анализ

Теперь заменим столбец price так, чтобы если в каком-то из столбцов встречается NaN, то заполняем новый столбец без их учетов

In [ ]:
cols = ['price_prod', 'price_sber']
doctors['price'] = doctors[cols].mean(axis=1, skipna=True)

#Корреляции (Heatmap)

##Корреляция Пирсона

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(doctors.corr(numeric_only = True), annot=True, cmap='RdBu_r')
plt.tight_layout()

Сильная корреляция наблюдается только между ценами на двух платформах, что подтверждает корректное сопоставление врачей при мердже
Между остальными переменными линейный связи не найдены.

**Вывод:** ключевые параметры в целом не имеют линейных зависимостей

##Корреляция Спирмена

- измеряет монотонную (возрастающую или убывающую, но не обязательно линейную)зависимость между переменными.

In [ ]:
corr = doctors.corr(method='spearman', numeric_only=True)


plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, fmt=".2f")
plt.title('Корреляции (Спирмен)')

plt.tight_layout()
plt.show()



**1. Опыт и рейтинг**

* Между опытом и рейтингом на СберЗдоровье умеренная положительная корреляция (0.6).
* На ПроДокторов слабая отрицательная (около –0.2).

  Вывод: опытные врачи оцениваются выше на Сбере, тогда как на ПроДокторов стаж почти не влияет на рейтинг

**2. Рейтинг и количество отзывов**

* На обеих платформах наблюдается слабая положительная корреляция (0.4): больше отзывов: немного выше рейтинг

**3. Связь между платформами**

* Количество отзывов слегка коррелирует между платформами (0.5).
* Цены сильно коррелируют (0.9)

**4. Количество клиник и отзывы**

* Между числом клиник и отзывов: умеренная положительная связь (0.5): врачи, работающие в нескольких клиниках, получают больше отзывов.

**5. Общие наблюдения**

* Присутствуют нелинейные корреляции
* Сильных корреляций, кроме ценовых, нет
* Основные связи наблюдаются между ценами, отзывами и рейтингами внутри одной платформы
* В остальном признаки слабо связаны, что говорит о неоднородности данных и различиях между платформами


#Стаж и цена

In [ ]:
plt.figure(figsize=(16, 9))

doctors_1 = doctors[doctors['price_prod'] < 9000]

sns.jointplot(x='experience', y='price_prod', data=doctors_1, kind='hex')

plt.suptitle('Сосредоточение данных по цене и стажу в Продокторов', y=1.02)
plt.show()

plt.tight_layout()

In [ ]:
doctors_1 = doctors[doctors['price'] < 9000]

In [ ]:
sns.jointplot(x='experience', y='price_prod', data=doctors_1, kind='hex',color = 'green')
plt.suptitle('Сосредоточение данных по цене и стажу на Сбере', y=1.02)
plt.show()

In [ ]:
sns.jointplot(x='experience', y='price', data=doctors_1, kind='hex',color = 'green')
plt.suptitle('Сосредоточение данных по цене и стажу в целом', y=1.02)
plt.show()

Нету роста стоимости приёма с увеличением стажа на обеих платформах

Основная масса врачей (5–25 лет опыта) концентрируется в диапазоне 2500–4000 ₽, при этом даже врачи с опытом более 30 лет не имеют существенно более высоких цен. Это указывает на слабую монетизацию опыта и унифицированное ценообразование внутри платформы.

### Рассмотрим связку цена, рейтинг, опыт

In [ ]:
sns.jointplot(x='experience', y='price', data=doctors_1, kind='hex',color = '#b977e6')
plt.suptitle('Сосредоточение данных по цене и стажу', y=1.02)
plt.show()

In [ ]:
sns.jointplot(x='experience', y='rating', data=doctors_1, kind='hex',color = '#b977e6')
plt.suptitle('Сосредоточение данных по рейтингу и опыту', y=1.02)
plt.show()

In [ ]:
d = doctors[pd.to_numeric(doctors['price'], errors='coerce').between(1, 13000)].copy()

sns.set_theme(style='whitegrid')
pp = sns.pairplot(
    d[['price','rating','experience']].dropna(),
    diag_kind='hist',
    plot_kws=dict(alpha=0.35, s=18, edgecolor='none'),
    diag_kws=dict(bins=20, edgecolor='none')
)

pp.fig.suptitle('Анализ взаимосвязи цены, рейтинга и опыта', y=1.02)
plt.show()

### Цена и рейтинг

In [ ]:
sns.jointplot(x='rating', y='price', data=d, kind='hex',color = '#b977e6')
plt.suptitle('Сосредоточение данных по рейтингу и цене', y=1.02)
plt.show()

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(d['rating'], d['price'], s=12, alpha=0.4, edgecolor='none')
plt.xlabel('Рейтинг'); plt.ylabel('Цена')
plt.title('Цена и рейтинг')
plt.grid(alpha=.3)
plt.tight_layout()
plt.show()

In [ ]:
s = d['experience']

q1 = s.quantile(0.25)
q3 = s.quantile(0.75)
iqr = q3 - q1
low  = q1 - 1.5*iqr
high = q3 + 1.5*iqr
high

In [ ]:
d = doctors[doctors['price'].between(1,13000)].dropna(subset=['experience','rating','price']).copy()
d['bin_exp'] = pd.cut(d['experience'], bins=np.arange(0, 61, 5), right=False) # берем бины, у нас выбросы начинаются с 58, но достроим график до 60, чтобы были равные бмны
d['bin_rat'] = pd.cut(d['rating'], bins=np.linspace(0,5,11), right=False) # 10 равных интервалов по 0.5

heat = (d.groupby(['bin_exp','bin_rat']).agg(mean_price=('price','mean'), n=('price','size')).reset_index())
pivot = heat.pivot(index='bin_rat', columns='bin_exp', values='mean_price')

plt.figure(figsize=(9,6))
graph = plt.imshow(pivot, aspect='auto', origin='lower', cmap='magma')
plt.colorbar(graph, label='Средняя цена')
plt.yticks(range(len(pivot.index)), [i.left for i in pivot.index])
plt.xticks(range(len(pivot.columns)), [c.left for c in pivot.columns])
plt.xlabel('Опыт'); plt.ylabel('Рейтинг')
plt.title('Средняя цена по опыту и рейтингу')
plt.tight_layout()
plt.show()

### Введем ценовые категории и просмотрим какие-либо зависимотсти

In [ ]:
labels = ['Бюджетный', 'Средний', 'Премиум']

doctors['price_tier'] = pd.qcut(doctors['price'], q=[0, 1/3, 2/3, 1], labels=labels)
doctors['price_tier'].value_counts()

In [ ]:
sns.set_theme(style='whitegrid')
sns.boxplot(data=doctors, x='price_tier', y='rating', showfliers=False)

plt.xlabel('Ценовая категория')
plt.ylabel('Рейтинг')
plt.title('Рейтинг по ценовым категориям')

plt.tight_layout()
plt.show()

In [ ]:
sns.boxplot(data=doctors, x='price_tier', y='rating', showfliers=False)
plt.xlabel('Ценовая категория')
plt.ylabel('Отзывы')
plt.title('Популярность и цена')

plt.tight_layout()
plt.show()

In [ ]:
sns.boxplot(data=doctors, x='price_tier', y='experience', showfliers=False)
plt.xlabel('Ценовая категория')
plt.ylabel('Опыт')
plt.title('Опыт по ценовым категориям')
plt.tight_layout()
plt.show()

### Портрет типичного врача

In [ ]:
df = doctors.copy()

def med(s):
    s = pd.to_numeric(s, errors='coerce').dropna()
    return np.percentile(s, 50) if len(s) else np.nan

profile_overall = {
    'experience': med(df.get('experience')),
    'rating': med(df.get('rating')),
    'price': med(df.get('price')),
}

print('Портрет типичного врача:')
print(f"Опыт: {profile_overall['experience']} лет")
print(f"Рейтинг: {profile_overall['rating']:.2f}")
print(f"Цена: {profile_overall['price']} руб.".replace(',', ' ') )


### Правда ли что популярные врачи дороже стоят?

In [ ]:
review_cols = ['review_count_sber','review_count_prod']
df['reviews_total'] = df[review_cols].fillna(0).sum(axis=1) if review_cols else np.nan

d = df[df['price'] <= 13000].dropna(subset=['price','reviews_total']).copy()
d = d[d['reviews_total'] <= 400]

plt.figure(figsize=(7,5))
plt.scatter(d['reviews_total'], d['price'], s=8, alpha=0.2, edgecolor='none')

plt.xlabel('Число отзывов')
plt.ylabel('Цена')
plt.legend(frameon=False)
plt.grid(alpha=.3)

plt.tight_layout()
plt.show()

Большинство врачей сконцентрировано в диапазоне 0–100 отзывов при широком разбросе цен (от 1 до 13 тыс). То есть при одинаковой популярности цена бывает любой. Популярность (число отзывов) почти не связана с ценой. При любом количестве отзывов встречаются как бюджетные, так и дорогие врачи.

### Есть ли  большие разрывы в оценке на разных платформах для одного и того же врача

In [ ]:
df['gap'] = df['rating_sber'] - df['rating_prod']
df['abs_gap'] = df['gap'].abs()

df['reviews_total'] = df[['review_count_sber','review_count_prod']].sum(axis=1, skipna=True) # общий счётчик отзывов

In [ ]:
# разрыв только где есть оба рейтинга
d = df.dropna(subset=['rating_sber','rating_prod']).copy()
d = d[(d['review_count_sber'] >= 5) & (d['review_count_prod'] >= 5)]
d['gap'] = d['rating_sber'] - d['rating_prod']
d['abs_gap'] = d['gap'].abs()

cols = ['name', 'rating_sber','review_count_sber', 'rating_prod','review_count_prod', 'gap','abs_gap']

top_gap = d.sort_values('abs_gap', ascending=False)[cols].head(20)
top_gap

In [ ]:
top_gap.shape[0]

Из таблицы видно, что завышенная оценка встречается в сберздоровье, в то время как у этих же врачей на продокторов рейтинг стремится к 0.

# Соханение результата

In [ ]:
doctors.to_csv('doctors_eda_result.csv', encoding='utf-8')